## Frame

In [1]:
using Gridap
using Plots
using GridapMakie
using CairoMakie 
using GLMakie
using Gridap.Geometry
using GridapGmsh
using Gmsh: Gmsh, gmsh

In [2]:
# Initialize the Gmsh API
gmsh.initialize()

# Open the .geo file
geo_file = "./msh_files/OFFSHORE_NASAL_JACKET.geo"
gmsh.open(geo_file)

# Set options to ensure lines are not subdivided
gmsh.option.set_number("Mesh.ElementOrder", 1)

# println("Mesh saved as $msh_file")

Info    : Reading './msh_files/OFFSHORE_NASAL_JACKET.geo'...
Info    : Done reading './msh_files/OFFSHORE_NASAL_JACKET.geo'


In [3]:
gmsh.option.set_number("Mesh.CharacteristicLengthMax", 1e3)
gmsh.option.set_number("Mesh.CharacteristicLengthMin", 1e2)

# Generate the mesh
gmsh.model.mesh.generate(1)  # Generate a 1D mesh (the physical elements are 1D (!))

# Save as .msh file
msh_file = "./msh_files/OFFSHORE_NASAL_JACKET.msh"
gmsh.write(msh_file)

# Finalize Gmsh
gmsh.finalize()

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 20%] Meshing curve 4 (Line)
Info    : [ 20%] Meshing curve 5 (Line)
Info    : [ 20%] Meshing curve 6 (Line)
Info    : [ 30%] Meshing curve 7 (Line)
Info    : [ 30%] Meshing curve 8 (Line)
Info    : [ 30%] Meshing curve 9 (Line)
Info    : [ 40%] Meshing curve 10 (Line)
Info    : [ 40%] Meshing curve 11 (Line)
Info    : [ 50%] Meshing curve 12 (Line)
Info    : [ 50%] Meshing curve 13 (Line)
Info    : [ 50%] Meshing curve 14 (Line)
Info    : [ 60%] Meshing curve 15 (Line)
Info    : [ 60%] Meshing curve 16 (Line)
Info    : [ 60%] Meshing curve 17 (Line)
Info    : [ 70%] Meshing curve 18 (Line)
Info    : [ 70%] Meshing curve 19 (Line)
Info    : [ 80%] Meshing curve 20 (Line)
Info    : [ 80%] Meshing curve 21 (Line)
Info    : [ 80%] Meshing curve 22 (Line)
Info    : [ 90%] Meshing curve 23 (Line)
Info    : [ 90%] Meshing curve 24 (Line)
I

In [4]:
# Define known parameters/functions for the rod equation

# Nacelle
F_Nacelle = 5000*9.81 # inwards tension N

# Pile 
m_Pile(x) = (1000*9.81) # N/m
EA_Pile = 1e8 # Nm2

# Jacket 
m_Jacket(x) = (100*9.81) # N/m 
EA_Jacket = 1e6 # Nm2

# E = 200e9 # Young's modulus (Pa)
# h = 0.10
# t = 0.10
# A = h*t # Cross-sectional area (m²)
# EA = E*A

# f(x) = 1000.0 # N/m
# T = 500e6 # N (tensile force acting on the middle and top element)

model = GmshDiscreteModel("msh_files/OFFSHORE_NASAL_JACKET.msh")

Info    : Reading 'msh_files/OFFSHORE_NASAL_JACKET.msh'...
Info    : 48 entities
Info    : 21 nodes
Info    : 31 elements
Info    : Done reading 'msh_files/OFFSHORE_NASAL_JACKET.msh'


UnstructuredDiscreteModel()

In [5]:
# Set up the figure and axis 
labels = get_face_labeling(model)
initial_tags = get_tag_entities(labels)[end]
tag_from_names = get_tag_from_name(labels)

Dict{String, Int64} with 5 entries:
  "Jacket Members" => 4
  "Pile Members"   => 5
  "Supports"       => 1
  "Bottom Pile"    => 3
  "Nacelle"        => 2

In [6]:
# Set up the figure and axis 
fig = Figure(resolution=(800, 600))
ax = Axis(fig[1, 1], title="Domain with Boundary Tags", xlabel="x", ylabel="y")

for intgr in length(tag_from_names)
    CairoMakie.wireframe!(ax, Triangulation(model))
end

CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Supports"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Nacelle"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Jacket Members"))
CairoMakie.scatter!(ax, Boundary(Triangulation(model), tags="Pile Members"))

fig

In [7]:
# add_tag_from_tags!(labels, "Dirichlet", [1]) # Our Dirichlet is the base (fixed displacement) (tag = 1)
# add_tag_from_tags!(labels, "Neumann", [2]); # Our Neumann BC is at the Nacelle (tag = 2)

In [8]:
Ω_pile = Triangulation(model, tags="Pile Members")
Ω_jacket = Triangulation(model, tags="Jacket Members")
Γ_D = Boundary(Ω_jacket,tags="Supports")
Γ_N = Boundary(Ω_pile,tags="Nacelle")

CompositeTriangulation()

In [9]:
order = 1
reffe_u = ReferenceFE(lagrangian, Float64, order) # We create a Reference Finite Element (it serves as a basis function on one single triangulation element (puzzle piece of the entire puzzle))

V = TestFESpace(Ω_jacket, reffe_u, dirichlet_tags="Supports") # The TestFESpace for the rod equation
U = TrialFESpace(V, 0.0) # The TrialFESpace for the rod equation

TrialFESpace()

In [10]:
degree = 2*order 
dΩ_Pile = Measure(Ω_pile, degree) # Numerical integration within the domain Ω using second-order Gaussian quadrature rule
dΩ_Jacket = Measure(Ω_jacket, degree) # Numerical integration within the domain Ω using second-order Gaussian quadrature rule
dΓ_N = Measure(Γ_N,degree) # Numerical integration along the Neumann boundary Γ_D using second-order Gaussian quadrature rule
nΓ_N = get_normal_vector(Γ_N) # Normal vector to the Neumann boundary Γ_N

GenericCellField():
 num_cells: 1
 DomainStyle: ReferenceDomain()
 Triangulation: CompositeTriangulation()
 Triangulation id: 2876907366653897562

In [11]:
# Weak form
a(u, v) = ∫(EA_Jacket * ∇(u) ⋅ ∇(v))dΩ_Jacket + ∫(EA_Pile * ∇(u) ⋅ ∇(v))dΩ_Pile
l(v) = ∫(m_Jacket * v)dΩ_Jacket + ∫(m_Pile * v)dΩ_Pile + ∫(F_Nacelle * v)dΓ_N

op = AffineFEOperator(a,l,U,V)

AffineFEOperator()

In [55]:
Ω = Triangulation(model) # The domain Ω is the entire mesh (jacket + pile)

uh = solve(op)
writevtk(Ω,"2d_frame",cellfields=["u"=>uh])

(["2d_frame.vtu"],)

In [59]:
l(v) = ∫(0.0*v)dΩ_Jacket + ∫(0.0*v)dΩ_Pile + ∫(0.0*v)dΓ_N

get_matrix(AffineFEOperator(a, l, U, V ))

9×9 SparseArrays.SparseMatrixCSC{Float64, Int64} with 37 stored entries:
      1.51157e5  -37446.2        …    ⋅               ⋅ 
 -37446.2             2.13902e5       ⋅               ⋅ 
 -56855.2        -32819.9             ⋅               ⋅ 
       ⋅         -56855.2             ⋅               ⋅ 
       ⋅               ⋅              ⋅               ⋅ 
       ⋅               ⋅         …    ⋅         -56855.2
       ⋅               ⋅            -1.23993e5        ⋅ 
       ⋅               ⋅             2.47985e5      -1.23993e5
       ⋅               ⋅            -1.23993e5       1.80848e5